In [ ]:
# remove first if exists
!rm -rf arabic-poetry-speech-classification.git
!git clone https://github.com/MagedSaeed/arabic-poetry-speech-classification.git

Cloning into 'arabic-poetry-speech-classification'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 49 (delta 21), reused 30 (delta 11), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [ ]:
# mount the drive to get the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# put the task path you want to work on here
%cd arabic-poetry-speech-classification/signal-classification

/content/arabic-poetry-speech-classification/signal-classification


In [ ]:
!pip install -q torchaudio
!pip install -q transformers
!pip install -q datasets
!pip install -q lang_trans
!pip install -q arabic_reshaper
!pip install -q python-bidi
!pip install -q pydub
!pip install -q soundfile
!pip install -q jiwer
!pip install -q PyArabic

     |████████████████████████████████| 3.5 MB 7.7 MB/s 
     |████████████████████████████████| 596 kB 68.0 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 6.8 MB 65.5 MB/s 
     |████████████████████████████████| 895 kB 60.5 MB/s 
     |████████████████████████████████| 311 kB 8.7 MB/s 
     |████████████████████████████████| 134 kB 86.7 MB/s 
     |████████████████████████████████| 1.1 MB 77.9 MB/s 
     |████████████████████████████████| 212 kB 84.7 MB/s 
     |████████████████████████████████| 144 kB 85.9 MB/s 
     |████████████████████████████████| 271 kB 88.3 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
     |████████████████████████████████| 126 kB 9.0 MB/s 


In [ ]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
from tqdm import tqdm
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
from trainer import CTCTrainer #1
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser 
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from models import Wav2Vec2ClassificationModel
from processors import CustomWav2Vec2Processor #2
from typing import Any, Dict, List, Optional, Union
from sklearn.model_selection import train_test_split
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from arg_parsers import DataTrainingArguments, ModelArguments, DataCollatorCTCWithPadding #3
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
% matplotlib inline

In [ ]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/dataset_wav.zip .

In [ ]:
if os.path.exists('dataset_wav'):
  if len(os.listdir('dataset_wav')) == 0:
    os.system('unzip dataset_wav.zip')
else:
  os.system('unzip dataset_wav.zip')

In [ ]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx'
test_metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx'
# dataset_folder = 'dataset'

In [ ]:
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx .

In [ ]:
dataset_wav_folder = 'dataset_wav'

In [ ]:
def preprocess_metadata(metadata):
  metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
  metadata['Clean Script'] = metadata['Script'].apply(lambda script: araby.strip_tashkeel(araby.strip_tatweel(script)))
  metadata['File Exist'] = metadata['Utterance name'].apply(lambda file_name: os.path.isfile(f'{dataset_wav_folder}/{file_name}'))
  metadata = metadata[metadata['File Exist'] == True]
  return metadata

In [ ]:
metadata = preprocess_metadata(pd.read_excel(metadata_path))
test_metadata = preprocess_metadata(pd.read_excel(test_metadata_path))
train_metadata = metadata[[script not in test_metadata['No.'].values for script in metadata['No.']]]
train_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Is_Malformed,Reciter_Label,Reciter_Gender,Further info on Bahr,Name of document,Clean Script,File Exist
0,1.0,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,0.0,0.0,0.0,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع ال...,True
1,2.0,P-2.aac,حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ من...,المديد,ديوان الطرماح ص227,2.aac,2.aac,0.0,0.0,0.0,NaN,NaN,حسرت عنه الرياح فأبدت منتأى كالقرو رهن ا...,True
2,3.0,P-3.aac,وخصيفَ اللَّونِ جادَتْ بهِ مَرْخَةٌ مِنْ ...,المديد,ديوان الطرماح ص227,3.aac,3.aac,0.0,0.0,0.0,NaN,NaN,وخصيف اللون جادت به مرخة من مخدج أو تمام,True
3,4.0,P-4.aac,بَيْنَ أظْآرٍ بِمَظْلُومَةٍ كَسَرَاةِ الس...,المديد,ديوان الطرماح ص227,4.aac,4.aac,0.0,0.0,0.0,NaN,NaN,بين أظآر بمظلومة كسراة الساق ساق الحمام,True
4,5.0,P-5.aac,مَنْزِلًا كَانَ لَنَا مَرَّةً وطنًا نحتلّ...,المديد,ديوان الطرماح ص227,5.aac,5.aac,0.0,0.0,0.0,NaN,NaN,منزلا كان لنا مرة وطنا نحتله كل عام,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,3731.0,P-3731.mp3,وَلِّ السَلاطينَ مَن تَوَلّاها وَاِلجَأ إ...,المنسرح,-,45.mp3,45.mp3,NaN,NaN,NaN,NaN,NaN,ول السلاطين من تولاها والجأ إليه تكن حدياها,True
3682,3732.0,P-3732.mp3,وَلا تَغُرَّنَّكَ الإِمارَةُ في غَيرِ أَم...,المنسرح,-,46.mp3,46.mp3,NaN,NaN,NaN,NaN,NaN,ولا تغرنك الإمارة في غير أمير وإن بها باهى,True
3683,3733.0,P-3733.mp3,فَإِنَّما المَلكُ رَبُّ مَملَكَةٍ قَد فَغ...,المنسرح,-,47.mp3,47.mp3,NaN,NaN,NaN,NaN,NaN,فإنما الملك رب مملكة قد فغم الخافقين سرياها,True
3684,3734.0,P-3734.mp3,مُبتَسِمٌ وَالوُجوهُ عابِسَةٌ سِلمُ العِد...,المنسرح,-,48.mp3,48.mp3,NaN,NaN,NaN,NaN,NaN,مبتسم والوجوه عابسة سلم العدى عنده كهيجاها,True


In [ ]:
test_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,89.0,P-89.aac,إنَّ بالشِّعبِ الَّذي دُونَ سَلْعٍ لَقتيل...,المديد,ديوان ثابت بن جابرص35,89.aac,89.aac,NaN,NaN,إن بالشعب الذي دون سلع لقتيلا دمه ما يطل,True
1,90.0,P-90.aac,خَلَّفَ الْعِبْءَ عَلَيَّ وَولَّى أنا بال...,المديد,ديوان ثابت بن جابرص35,90.aac,90.aac,NaN,NaN,خلف العبء علي وولى أنا بالعبء له مستقل,True
2,91.0,P-91.aac,ووراءَ الثَّأر منِّي ابنُ أختٍ مَصِعٌ عُق...,المديد,ديوان ثابت بن جابرص35,91.aac,91.aac,NaN,NaN,ووراء الثأر مني ابن أخت مصع عقدته ما تحل,True
3,92.0,P-92.aac,مُطْرِقٌ يَرْشَحُ سَمًّا كَمَا أَطْ رَقَ ...,المديد,ديوان ثابت بن جابرص35,92.aac,92.aac,NaN,NaN,مطرق يرشح سما كما أط رق أفعى ينفث السم صل,True
4,93.0,P-93.aac,خبرٌ ما نابَنا مُصْمَئِلٌّ جلَّ حتّى دقَّ...,المديد,ديوان ثابت بن جابرص35,93.aac,93.aac,NaN,NaN,خبر ما نابنا مصمئل جل حتى دق فيه الأجل,True
...,...,...,...,...,...,...,...,...,...,...,...
354,3682.0,P-3682.m4a,حِسانُ الوُجوهِ حِدادُ السُيو فِ يَبتَدِر...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (98).m4a,التسجيل (98).m4a,NaN,NaN,حسان الوجوه حداد السيو ف يبتدر المجد شبانها,True
355,3683.0,P-3683.m4a,وَبِالشَوطِ مِن يَثرِبٍ أَعبُدٌ سَتَهلِكُ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (99).m4a,التسجيل (99).m4a,NaN,NaN,وبالشوط من يثرب أعبد ستهلك في الخمر أثمانها,True
356,3684.0,P-3684.m4a,يَهونُ عَلى الأَوسِ أَثمانُهُم إِذا راحَ ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (100).m4a,التسجيل (100).m4a,NaN,NaN,يهون على الأوس أثمانهم إذا راح يخطر نشوانها,True
357,3685.0,P-3685.m4a,أَتَتهُم عَرانينُ مِن مالِكٍ سِراعٌ إِلى ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (101).m4a,التسجيل (101).m4a,NaN,NaN,أتتهم عرانين من مالك سراع إلى الروع فتيانها,True


In [ ]:
# metadata_test_path = 'testset.csv'
# metadata_train_path = 'trainset.csv'
# metadata_val_path = 'valset.csv'
# dataset_folder = 'dataset'
# dataset_wav_folder = 'dataset_wav'

In [ ]:
# train_metadata = pd.read_csv(metadata_train_path)
# test_metadata = pd.read_csv(metadata_test_path)
# val_metadata = pd.read_csv(metadata_val_path)
train_metadata,val_metadata = train_test_split(train_metadata, test_size=0.1,random_state=45)

In [ ]:
# sample_rates = set()
# for file_path in tqdm(itertools.chain(
#     train_metadata['Utterance name'],
#     test_metadata['Utterance name'],
#     val_metadata['Utterance name']
#   )):
#   complete_path = f'{dataset_folder}/{file_path}'
#   complete_wav_path = f'{dataset_wav_folder}/{file_path}'
#   # os.system(f'ffmpeg -i {complete_path} {complete_wav_path}')
#   audio = AudioSegment.from_file(complete_path)
#   sample_rates.add(audio.frame_rate)
#   audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
# sample_rates

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
if is_apex_available():
    from apex import amp

In [ ]:
if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast

logger = logging.getLogger(__name__)

## getting training arguments

In [ ]:
# testing path poetry-classification
# model_output_path = '--output_dir=poetry-classification'
args_string="""
--model_name_or_path="bakrianoo/sinai-voice-ar-stt"
--output_dir=/content/drive/MyDrive/KFUPM-Master/ICS606/Models/NoShuffledDatasetV2
--freeze_feature_extractor 
--num_train_epochs="3" 
--per_device_train_batch_size="2" 
--preprocessing_num_workers="1" 
--learning_rate="3e-5" 
--warmup_steps="5" 
--evaluation_strategy="steps"
--save_steps="500" 
--eval_steps="150" 
--save_total_limit="1" 
--logging_steps="100"
--weight_decay=1e-1
--do_eval=1
--do_train=1
"""
def match_numeric_type(s):
  try:
    return json.loads(s)
  except:
    return s

# construct the args dict from the args string
args_dict = dict()
for line in args_string.splitlines():
  line = line.replace('--','').replace(r'\\','').replace('"','').replace("'",'').strip()
  if line:
    if '=' in line:
      arg_name,arg_value = line.split('=')
      args_dict[arg_name] = arg_value
    else:
      args_dict[line] = ''

# match args to dataclasses
args_class_attrs = defaultdict(list)
for arg_name,arg_value in args_dict.items():
  for ArgClass in (ModelArguments, DataTrainingArguments, TrainingArguments):
    if arg_name in ArgClass.__annotations__:
      args_class_attrs[ArgClass].append(arg_name)

# initialzie args objects
model_args = ModelArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[ModelArguments]})
data_args = DataTrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[DataTrainingArguments]})
training_args = TrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[TrainingArguments]})

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


## logging setup

In [ ]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
# logger.setLevel(
#     logging.INFO if is_main_process(training_args.local_rank) else logging.WARN
# )
logger.setLevel(
    logging.WARN
)
# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
# Set the verbosity to info of the Transformers logger (on main process only):
if is_main_process(training_args.local_rank):
    transformers.utils.logging.set_verbosity_info()

# Set seed before initializing model.
set_seed(training_args.seed)


02/18/2022 19:41:17 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False


## dataset processing

In [ ]:
# train_metadata = pd.read_csv(metadata_train_path)
# val_metadata = pd.read_csv(metadata_val_path)
# test_metadata = pd.read_csv(metadata_test_path)

train_dataset = Dataset.from_pandas(train_metadata)
eval_dataset = Dataset.from_pandas(val_metadata)
test_dataset = Dataset.from_pandas(test_metadata)

if data_args.max_train_samples is not None:
    train_dataset = train_dataset.select(range(data_args.max_train_samples))

if data_args.max_val_samples is not None:
    eval_dataset = eval_dataset.select(range(data_args.max_val_samples))
# Preprocessing the datasets.
# We need to read the aduio files as arrays and tokenize the targets.
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    # 32000: torchaudio.transforms.Resample(32000, 16000),
}

labels = {
    bahr: bahr_index
    for bahr_index, bahr in enumerate(sorted(set(train_metadata["Bahr"])))
}
print("labels are:", labels)
print("len:", len(labels))

def speech_file_to_array_fn(batch):
    start = 0
    stop = 20
    srate = 16_000
    speech_array, sampling_rate = torchaudio.load(
        f'dataset_wav/{batch["Utterance name"]}'
    )
    speech_array = speech_array[0]
    batch["speech"] = resamplers[sampling_rate](speech_array).squeeze().numpy()
    batch["sampling_rate"] = srate
    batch["parent"] = labels[batch["Bahr"]]
    return batch

train_dataset = train_dataset.map(
    speech_file_to_array_fn,
    remove_columns=train_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    speech_file_to_array_fn,
    remove_columns=eval_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

test_dataset = test_dataset.map(
    speech_file_to_array_fn,
    remove_columns=test_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,

)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16_000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True,
)
processor = CustomWav2Vec2Processor(feature_extractor=feature_extractor)

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    batch["input_values"] = processor(
        batch["speech"], sampling_rate=batch["sampling_rate"][0]
    ).input_values
    batch["labels"] = batch["parent"]
    return batch

train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    prepare_dataset,
    remove_columns=eval_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)
test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

labels are: {'البسيط': 0, 'الخفيف': 1, 'الرجز': 2, 'الرمل': 3, 'السريع': 4, 'الطويل': 5, 'الكامل': 6, 'المتدارك': 7, 'المتقارب': 8, 'المجتث': 9, 'المديد': 10, 'المضارع': 11, 'المقتضب': 12, 'المنسرح': 13, 'الهزج': 14, 'الوافر': 15}
len: 16


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

  0%|          | 0/1489 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:166: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/166 [00:00<?, ?ba/s]

  0%|          | 0/180 [00:00<?, ?ba/s]

In [ ]:
train_dataset[0]['speech']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

## prepare the model

In [ ]:
model = Wav2Vec2ClassificationModel.from_pretrained(
    "bakrianoo/sinai-voice-ar-stt",
    attention_dropout=0.01,
    hidden_dropout=0.01,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.01,
    gradient_checkpointing=True,
    num_attention_heads=4,
)

if model_args.freeze_feature_extractor:
    model.freeze_feature_extractor()

def compute_metrics(pred):
    labels = pred.label_ids.argmax(-1)
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    matrix = confusion_matrix(labels, preds)
    print(matrix)
    print('clasification report',report)
    print('accuracy',acc)
    return {"accuracy": acc}

# Data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# Initialize our Trainer
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=processor.feature_extractor,
)

https://huggingface.co/bakrianoo/sinai-voice-ar-stt/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpzu4u1g5d


Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

storing https://huggingface.co/bakrianoo/sinai-voice-ar-stt/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/36a5276e59d1697b39ffceae505fe4ebb3dda58f2489224516653bdd1b3d5c10.3f68d08c0b086036bc07fbca208c3d929c6d6e85b88b27094575c7febf00210f
creating metadata file for /root/.cache/huggingface/transformers/36a5276e59d1697b39ffceae505fe4ebb3dda58f2489224516653bdd1b3d5c10.3f68d08c0b086036bc07fbca208c3d929c6d6e85b88b27094575c7febf00210f
loading configuration file https://huggingface.co/bakrianoo/sinai-voice-ar-stt/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/36a5276e59d1697b39ffceae505fe4ebb3dda58f2489224516653bdd1b3d5c10.3f68d08c0b086036bc07fbca208c3d929c6d6e85b88b27094575c7febf00210f
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` i

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

storing https://huggingface.co/bakrianoo/sinai-voice-ar-stt/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/e9d64a122363cfa9bda415e9f40f5aba2c7fa30be88a873c6a46ec86f1a11751.7d16b678b1a8489f456457ae16ce87344f99a10b003f3ad534a173fff23631c9
creating metadata file for /root/.cache/huggingface/transformers/e9d64a122363cfa9bda415e9f40f5aba2c7fa30be88a873c6a46ec86f1a11751.7d16b678b1a8489f456457ae16ce87344f99a10b003f3ad534a173fff23631c9
loading weights file https://huggingface.co/bakrianoo/sinai-voice-ar-stt/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/e9d64a122363cfa9bda415e9f40f5aba2c7fa30be88a873c6a46ec86f1a11751.7d16b678b1a8489f456457ae16ce87344f99a10b003f3ad534a173fff23631c9
Some weights of the model checkpoint at bakrianoo/sinai-voice-ar-stt were not used when initializing Wav2Vec2ClassificationModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ClassificationModel from the che

## check for previous checkpoints

## training

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if (
    os.path.isdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

In [ ]:
# Training
if training_args.do_train:
  if last_checkpoint is not None:
      checkpoint = last_checkpoint
  elif os.path.isdir(model_args.model_name_or_path):
      checkpoint = model_args.model_name_or_path
  else:
      checkpoint = None
  train_result = trainer.train(resume_from_checkpoint=checkpoint)
  trainer.save_model()

  # save the feature_extractor and the tokenizer
  if is_main_process(training_args.local_rank):
      processor.save_pretrained(training_args.output_dir)

  metrics = train_result.metrics
  max_train_samples = (
      data_args.max_train_samples
      if data_args.max_train_samples is not None
      else len(train_dataset)
  )
  metrics["train_samples"] = min(max_train_samples, len(train_dataset))

  trainer.log_metrics("train", metrics)
  trainer.save_metrics("train", metrics)
  trainer.save_state()

# Evaluation
results = {}
if training_args.do_eval:
  logger.info("*** Evaluate ***")
  metrics = trainer.evaluate()
  max_val_samples = (
      data_args.max_val_samples
      if data_args.max_val_samples is not None
      else len(eval_dataset)
  )
  metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

  trainer.log_metrics("eval", metrics)
  trainer.save_metrics("eval", metrics)

Loading model from /content/drive/MyDrive/KFUPM-Master/ICS606/Models/NoShuffledDataset/checkpoint-2000).
***** Running training *****
  Num examples = 2978
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 4467
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 2000
  Will skip the first 1 epochs then the first 511 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/511 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss,Accuracy
2100,0.624800,0.790812,0.830816
2200,0.368200,0.817460,0.809668
2300,0.465900,1.099316,0.818731
2400,0.432800,0.682384,0.845921
2500,0.406200,0.726648,0.845921
2600,0.346400,0.701655,0.830816
2700,0.471000,0.701912,0.861027
2800,0.446400,0.733758,0.851964
2900,0.498400,0.537807,0.845921
3000,0.222600,0.627121,0.861027


***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


...

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


[[  6   0   0   1   0   1   2   0   0   0   0   0   0   0   0   0]
 [  1  13   0   0   0   2   0   0   0   1   0   0   0   0   0   0]
 [  0   0   2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  32   0   0   0   0   1   0   0   0   0   0   1   0]
 [  0   2   0   0  13   0   0   0   0   0   0   0   0   1   0   0]
 [  1   0   0   0   1 107   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   6  21   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   0   0  11   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   2   0   0   1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   7   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   1   0   0   0   0   0   0  17   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0]
 [  1   0   0   0   1   1   0   0   0   0   0   0   0  22   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0   0   0   0   6 

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


[[  7   0   0   0   0   1   2   0   0   0   0   0   0   0   0   0]
 [  1  11   0   1   0   2   0   1   0   1   0   0   0   0   0   0]
 [  0   0   2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  32   0   0   0   0   1   0   0   0   0   0   1   0]
 [  0   2   0   0  13   0   0   0   0   0   0   0   0   1   0   0]
 [  1   0   0   1   0 101   2   0   0   0   0   0   0   0   0   4]
 [  0   0   0   0   0   3  22   0   0   0   0   0   0   0   0   3]
 [  0   0   0   0   0   0   0  12   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   2   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   7   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   1   0   0   0   0   0   0  17   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0]
 [  1   0   0   0   1   1   0   0   0   0   0   0   0  22   0   0]
 [  0   0   0   1   0   0   0   3   0   0   0   0   0   0   3 

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive',force_remount=True)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
preds = trainer.predict(test_dataset=test_dataset)